# prediction timetracking for o_world data 

1. Manipulate data with ndarray.
2. Create a neural network.
3. Automatic differentiation with autograd.
4. Train the neural network.
5. Predict with a pre-trained model.
6. Use GPUs.


In [1]:
import torch

print(torch.__version__)

#파이토치 다운그레이드  
#conda install pytorch==1.5.1 torchvision==0.6.1 -c pytorch


1.8.1+cu102


In [2]:
!pip list | grep transformers
!pip list | grep torch
!pip list | grep numpy

transformers           4.2.2
pytorch-lightning      1.2.3
torch                  1.8.1
torchmetrics           0.3.2
torchvision            0.6.0a0+35d732a
numpy                  1.20.2


In [3]:
import torch
from torch import nn, Tensor
from torch.optim import Optimizer
from torch.utils.data import DataLoader, RandomSampler, DistributedSampler, random_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn import CrossEntropyLoss
from pytorch_lightning.core.lightning import LightningModule 
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.metrics.functional import accuracy, precision, recall
from transformers.models.bert.modeling_bert import BertModel
from transformers import AdamW, BertForSequenceClassification, AdamW, BertConfig, AutoTokenizer, BertTokenizer
from keras.preprocessing.sequence import pad_sequences


import random
import numpy as np 

import time
import datetime
import pandas as pd
import os
from tqdm import tqdm

#ImportError: cannot import name 'SAVE_STATE_WARNING'
#SAVE_STATE_WARNING from PyTorch, only exists in 1.5.0 
#downgrade pytorch version to 1.5.0

In [4]:
from torch.optim.lr_scheduler import ExponentialLR
import re
import emoji
from soynlp.normalizer import repeat_normalize
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from pprint import pprint

In [5]:
if torch.cuda.is_available():    

    # 파이토치에 GPU 할당    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2070


# 모델과 토크나이저

In [6]:
#토크나이저: kcbert
from transformers import AutoModelWithLMHead
model_name = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#bert = AutoModelWithLMHead.from_pretrained("beomi/kcbert-base")
model = BertForSequenceClassification.from_pretrained("beomi/kcbert-base")
#model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2) #label 수를 조정

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

# 데이터 로드 후 콘텐츠만 추출

In [8]:
df = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [9]:
train_set = df[0:120000]
valid_set = df[120000:135000]
test_set  = df[135000:150000]

In [10]:
print(len(train_set))
print(len(valid_set))
print(len(test_set))

120000
15000
15000


In [11]:
class TheDataset(torch.utils.data.Dataset):

    def __init__(self, document, label, tokenizer):
        self.document    = document
        self.label = label
        self.tokenizer  = tokenizer
        self.max_len    = tokenizer.model_max_length
  
    def __len__(self):
        return len(self.document)
  
    def __getitem__(self, index):
        document = str(self.document[index])
        label = self.label[index]

        encoded_review = self.tokenizer.encode_plus(
            document,
            add_special_tokens    = True,
            max_length            = self.max_len,
            return_token_type_ids = False,
            return_attention_mask = True,
            return_tensors        = "pt",
            padding               = "max_length",
            truncation            = True
        )

        return {
            'input_ids': encoded_review['input_ids'][0],
            'attention_mask': encoded_review['attention_mask'][0],
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [12]:
train_set_dataset = TheDataset(
    document    = train_set.document.tolist(),
    label = train_set.label.tolist(),
    tokenizer  = tokenizer,
)

valid_set_dataset = TheDataset(
    document    = valid_set.document.tolist(),
    label = valid_set.label.tolist(),
    tokenizer  = tokenizer,
)

# Create DataLoader for train/validation sets.
train_set_dataloader = torch.utils.data.DataLoader(
    train_set_dataset,
    batch_size  = 16,
    num_workers = 4
)

valid_set_dataloader = torch.utils.data.DataLoader(
    valid_set_dataset,
    batch_size  = 16,
    num_workers = 4
)

# Get one batch as example.
train_data = next(iter(train_set_dataloader))
valid_data = next(iter(valid_set_dataloader))

# Print the output sizes.
print( train_data["input_ids"].size(), valid_data["input_ids"].size() )

torch.Size([16, 300]) torch.Size([16, 300])


# model freeze

In [13]:
for name, param in model.bert.named_parameters():
    param.requires_grad = False

By running the above code, you are going through all the parameters and set its requires_grad attribute to zero. It means Huggingface will not try to optimize these weights.

In [18]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [27]:
import torch
from transformers import Trainer

training_args = TrainingArguments(
    output_dir                  = "./sentiment-analysis",
    num_train_epochs            = 1,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size  = 32,
    warmup_steps                = 500,
    weight_decay                = 0.01,
#    save_strategy               = "epoch",
    evaluation_strategy         = "steps"
)

In [25]:
# Define the Huggingface Trainer object
trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = train_set_dataset,
    eval_dataset    = valid_set_dataset,
    compute_metrics = compute_metrics
)


In [28]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
500,0.682600,0.640213,0.693600,0.660561,0.741748,0.595393,201.246300,74.536000


TrainOutput(global_step=938, training_loss=0.6562134634965519, metrics={'train_runtime': 1863.4143, 'train_samples_per_second': 0.503, 'total_flos': 23526734256000000, 'epoch': 1.0})

# Load a Checkpoint and get Predictions

In [33]:
# Load the checkpoint
model = BertForSequenceClassification.from_pretrained("./sentiment-analysis/checkpoint-500")

# Make the test set ready
test_set_dataset = TheDataset(
    document    = test_set.document.tolist(),
    label = test_set.label.tolist(),
    tokenizer  = tokenizer,
)

training_args = TrainingArguments(
    output_dir = "./sentiment-analysis",
    do_predict = True
)

trainer = Trainer(
    model           = model,
    args            = training_args,
    compute_metrics =compute_metrics,
)

trainer.predict(test_set_dataset)

PredictionOutput(predictions=array([[ 0.39245537,  0.06493665],
       [ 0.23860359,  0.18273208],
       [-0.1708882 ,  0.22821526],
       ...,
       [ 0.16608961, -0.0852609 ],
       [-0.01487223,  0.12979108],
       [-0.1741466 ,  0.1819702 ]], dtype=float32), label_ids=array([0, 0, 1, ..., 0, 1, 0]), metrics={'eval_loss': 0.638210117816925, 'eval_accuracy': 0.6956, 'eval_f1': 0.6588973554459883, 'eval_precision': 0.7375815353738083, 'eval_recall': 0.5953827460510328, 'eval_runtime': 192.1198, 'eval_samples_per_second': 78.076})

In [34]:
def calc_net_weight_count(net):
    net.train()
    net_params = filter(lambda p: p.requires_grad, net.parameters())
    weight_count = 0
    for param in net_params:
        weight_count += np.prod(param.size())
    return weight_count

print( "Number of Trainable Paramethers:", calc_net_weight_count(model) )

Number of Trainable Paramethers: 108920066


In [53]:
# Load the checkpoint
model = BertForSequenceClassification.from_pretrained("./sentiment-analysis/checkpoint-500")

In [81]:
device = "cuda:0"
model = model.to(device)

In [61]:
#DATA LOAD
column_name = ['date', 'description', 'contents', 'name']
df = pd.read_csv('/home/aiffel-dj57/project/오월드 여행 대전.csv')
df = df.values.tolist()
dataset = pd.DataFrame(df, columns=column_name)
dataset

,date,description,contents,name
0,20.01.31,"아, 그리고 대전 오월드 버드월드? 만들어져있다해서 거기도 한번 방문해야겠어여!대전...",\n\n대전 오월드 :: 대전 동물원 :: 대전 오월드 입장권 :: 대전 오월드 입...,김씨부부네 깨소금네 나는 블로그
1,15.03.11,대전여행중 아이와 갈만한곳 오월드 대전을 가긴갔다! 근데.. 아는곳이 없으니.. 신...,\n\n 대전여행중 아이와 갈만한곳 오월드 대전을 가긴갔다!근데.. 아는곳이 ...,메리킴
2,14.04.23,[대전여행] 대전 오월드 플라워랜드튤립축제 저는 어릴 적 튤립을 처음보고 “세상에 ...,\n\n \n\n[대전여행] 대전 오월드 플라워랜드튤립축제저는 어릴 적 튤립을 처음...,행복을 디자인하는 집
3,12.07.15,오 오 (무서울때 하는표현) 하더라고요~^^ ［대전 동물원/대전 오월드/대전동물원 ...,\n\n \n \n［대전동물원/대전 오월드/대전동물원주변맛집］주말당일여행\n \n지...,램프의 지니
4,16.10.23,대전 1박2일 대중교통 이용해서 여행 기차 여행 가보세용 당일치기로도 좋을 듯. 1...,\n\n대전 1박2일 대중교통 이용해서 여행기차 여행 가보세용당일치기로도 좋을 듯....,zoomin factory
...,...,...,...,...
2217,14.02.14,"대전여행에서 첫번째 들렀던 곳, 대전 오월드 포스팅 먼저 해볼게요! 타지역 여행가면...","\n\n\n\n\n대전여행에서 첫번째 들렀던 곳, 대전 오월드 포스팅 먼저 해볼게요...",♥지야월드♥
2218,13.10.15,"대전 오월드, 2013 개인적으로 활동하고 있는 사진 동호회에서 대전 오월드로 출사...",\n\n ...,도미노의 지극히 개인적인 생각
2219,14.08.08,"즐기다가 온 것 같아요. 대전으로 여행간이유는,- 대학교 1학년때부터 친한 친구가 ...",\n\n\n \n \n \n \n안뇽하세요 :) 소봉입니다.\n얼마전에 친구들이랑 ...,소봉이의 데일리 달링♥
2220,14.10.01,"축하해주러 대전으로 고고~ 열심히 박수치며 축하해주고, 끝나고 친구 가족들과 오월드...",\n\n2014.09.20.여보의 친구의 아기의 돌잔치를 축하해주러 대전으로 고고~...,결국엔 해피엔딩♡


In [62]:
train_set = dataset[0:1700]
valid_set = dataset[1700:1961]
test_set  = dataset[1961:2222]

In [63]:
print(len(train_set))
print(len(valid_set))
print(len(test_set))

1700
261
261


In [71]:
sentences = dataset['contents']
sentences

0       \n\n대전 오월드 :: 대전 동물원 :: 대전 오월드 입장권 :: 대전 오월드 입...
1       \n\n 대전여행중 아이와 갈만한곳 오월드    대전을 가긴갔다!근데.. 아는곳이 ...
2       \n\n \n\n[대전여행] 대전 오월드 플라워랜드튤립축제저는 어릴 적 튤립을 처음...
3       \n\n \n \n［대전동물원/대전 오월드/대전동물원주변맛집］주말당일여행\n \n지...
4       \n\n대전 1박2일 대중교통 이용해서 여행기차 여행 가보세용당일치기로도 좋을 듯....
                              ...                        
2217    \n\n\n\n\n대전여행에서 첫번째 들렀던 곳, 대전 오월드 포스팅 먼저 해볼게요...
2218    \n\n                                          ...
2219    \n\n\n \n \n \n \n안뇽하세요 :) 소봉입니다.\n얼마전에 친구들이랑 ...
2220    \n\n2014.09.20.여보의 친구의 아기의 돌잔치를 축하해주러 대전으로 고고~...
2221    \n\n< 가을 나들이 - '국화 축제' 대전 오월드 플라워랜드 >\n \n \nI...
Name: contents, Length: 2222, dtype: object

In [72]:
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 64

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [79]:
# 문장 테스트
def test_sentences(sentences):
 
    # 평가모드로 변경!!!!!
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [82]:
logits = test_sentences(['아이가 정말 곰이 됐다.'])

print(logits)
print(np.argmax(logits))

[[-0.22496635 -0.09296724]]
1


In [89]:
from torchtext.data import Iterator
batch_size = 32
train_loader = Iterator(sentences, batch_size = batch_size)

ImportError: cannot import name 'Iterator' from 'torchtext.data' (/home/aiffel-dj57/anaconda3/envs/aiffel/lib/python3.7/site-packages/torchtext/data/__init__.py)

In [85]:
val_iter= sentences.BucketIterator.splits(
        (sentences), batch_size=BATCH_SIZE,
        repeat=False)

AttributeError: 'Series' object has no attribute 'BucketIterator'

In [83]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy